In [1]:
library(data.table)
library(dplyr)
library(redeemR)
library(stringr)
library(BuenColors)
library(ggtree)
library(ape)
library(phangorn)
library(ggtreeExtra)
source('00_functions.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“replacing previous import ‘ape::where’ by ‘dplyr::where’ when loading ‘redeemR’”
Warning message:
“replacing previous import ‘ape::keep.tip’ by ‘tidytree::keep.tip’ when loading ‘redeemR’”
Warning message:
“replacing previous import ‘ape::drop.tip’ by ‘tidytree::drop.tip’ when loading ‘redeemR’”
Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: ggplot2

ggtree v3.12.0 For help: https://yulab-smu.top/treedata-book/

If you use the ggtree package suite in published research, please cite
the appropriate paper(s):

Guangchuang Yu, David Smith, Huachen Zhu, Yi Guan, Tommy Tsan-Yuk Lam.
g

In [ ]:

lapply(c("Young1.T1.BMMC", "Youn2.BMMC","Old1.BMMC", "Old2.BMMC"), function(x){ #, 
  analyze_connectivity_impact(x)
}) %>% rbindlist() %>% data.frame() -> pct_df1
pct_df1 <- pct_df1 

df_data <- pct_df1[,c("id", "perc1", "perc2", "perc3p")] %>%
  reshape2::melt(id.vars = c("id"))
df_data$variable <- factor(df_data$variable, levels = rev(c("perc3p", "perc2", "perc1")))
df_data$id <- factor(df_data$id, levels = (c( "Old1.BMMC", "Old2.BMMC","Young1.T1.BMMC", "Youn2.BMMC")))

pW <-  df_data %>%
  ggplot(aes(x = id, y = value, fill = variable)) + 
  geom_bar(stat = "identity", color = "black", width = 0.8, position = "dodge") + 
  scale_fill_manual(values = rev(c("lightgrey", "grey", "darkgrey"))) + 
  pretty_plot(fontsize = 7) + L_border() + 
  theme(legend.position = "none")+ 
  scale_y_continuous(expand = c(0,0), limits = c(0, 100)) + 
  labs(x = "", y = "% of cell / variant calls")
pW
cowplot::ggsave2(pW, file = "../final_plots/pct_1.pdf", width = 2.8, height = 2)



In [4]:
id = "Old2.BMMC"
base_dir = '/Users/tgao/redeem/'

print(id)
WD <-  paste0(base_dir, id, ".Consensus.final/") # CL specific working directory

# Import data based on reproducibility notebook
redeemR<-Create_redeemR(redeemR.read(path=WD,thr="S",Processed=F,rdsname = "/new.VariantsGTSummary.RDS"))
redeemR@HomoVariants <- (redeemR@V.fitered %>% filter(totalVAF > 0.35) %>% pull(Variants)) # Variants in Old 2 that are at 40%

(redeemR@V.fitered %>% filter(totalVAF > 0.35) %>% filter(HomoTag != "Homo"))

# Append the variants that Chen manually filters out already as homoplasmic
# so they aren't in the matrix
redeemR@HomoVariants <- c(redeemR@HomoVariants, bad_vars_cw)

# Create matrix using redeem functions
redeemR <- Make_matrix(redeemR)

## Filter low coverage cells
BadCells<-subset(redeemR@CellMeta,meanCov<10)$Cell
keep_cells <- !(rownames(redeemR@Cts.Mtx.bi) %in% BadCells)

# Count the evidence per mutation
mat <- redeemR@Cts.Mtx[keep_cells,]

# Summary statistics
data.frame(id, what = "redeem",
         total = sum(mat>0),
         n1 = sum(mat == 1),
         n2 = sum(mat == 2),
         n3 = sum(mat == 3))%>%
mutate(perc1 = n1/total*100) %>%
mutate(perc2 = n2/total*100) %>% mutate(perc3p = 100 - perc1 - perc2)

[1] "Old2.BMMC"


By default only total depth is summarized

deprecated



[1] "Use all cells"
[1] "12929 Cells in QualifiedTotalCts"
[1] "44087 variants to start"
[1] "43732 variants after remove low quality cells"
 [1] "Tag Homoplasmy: 10463_T_C" "Tag Homoplasmy: 11251_A_G"
 [3] "Tag Homoplasmy: 11719_G_A" "Tag Homoplasmy: 11812_A_G"
 [5] "Tag Homoplasmy: 11914_G_A" "Tag Homoplasmy: 13368_G_A"
 [7] "Tag Homoplasmy: 13711_G_A" "Tag Homoplasmy: 14233_A_G"
 [9] "Tag Homoplasmy: 1438_A_G"  "Tag Homoplasmy: 14766_C_T"
[11] "Tag Homoplasmy: 14905_G_A" "Tag Homoplasmy: 15326_A_G"
[13] "Tag Homoplasmy: 15452_C_A" "Tag Homoplasmy: 15607_A_G"
[15] "Tag Homoplasmy: 15928_G_A" "Tag Homoplasmy: 16126_T_C"
[17] "Tag Homoplasmy: 16294_C_T" "Tag Homoplasmy: 16325_T_C"
[19] "Tag Homoplasmy: 16519_T_C" "Tag Homoplasmy: 1888_G_A" 
[21] "Tag Homoplasmy: 263_A_G"   "Tag Homoplasmy: 2706_A_G" 
[23] "Tag Homoplasmy: 3666_G_A"  "Tag Homoplasmy: 4216_T_C" 
[25] "Tag Homoplasmy: 4769_A_G"  "Tag Homoplasmy: 4917_A_G" 
[27] "Tag Homoplasmy: 5333_T_C"  "Tag Homoplasmy: 7028_C_T" 
[29] 

Variants,CellN,PositiveMean,maxcts,CellNPCT,TotalVcount,TotalCov,totalVAF,CV,HomoTag
<chr>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
10810_T_C,7298,0.5628405,311,0.7976828,182953,464300,0.3940405,0.25610082,Hetero
310_T_C,8290,0.6277797,93,0.9061100,43434,98824,0.4395086,0.08037992,Hetero
72_T_C,8809,0.9163483,376,0.9628375,193386,242607,0.7971163,0.04699055,Hetero


Loading required package: Matrix.utils

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


The following object is masked from ‘package:ggtree’:

    expand


Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



id,what,total,n1,n2,n3,perc1,perc2,perc3p
<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
Old2.BMMC,redeem,88290,71629,4299,1966,81.12923,4.869181,14.00159
